
Las inyecciones representan una de las vulnerabilidades más comunes y peligrosas en las aplicaciones web. Consisten en la inserción de código malicioso o no deseado en una aplicación vulnerable, con el objetivo de alterar su funcionamiento previsto. Este tipo de ataque puede ocurrir cuando una aplicación no valida o desinfecta correctamente los datos introducidos por los usuarios, lo que permite a un atacante ejecutar comandos maliciosos, obtener acceso no autorizado a datos, o incluso comprometer la seguridad del sistema.

## INYECCIONES LDAP

### Descripción

Una inyección LDAP (Lightweight Directory Access Protocol) es un tipo de ataque que aprovecha vulnerabilidades en las consultas LDAP de una aplicación para manipular o modificar los datos enviados a un servidor de directorio. Al igual que las inyecciones SQL, las inyecciones LDAP ocurren cuando las entradas de usuario no se validan ni desinfectan adecuadamente, permitiendo que un atacante inserte comandos o filtros LDAP maliciosos en las consultas.

El protocolo LDAP se utiliza comúnmente para acceder y gestionar información en directorios como Active Directory, que contienen datos sobre usuarios, dispositivos y otros recursos de red. Un atacante puede aprovechar una vulnerabilidad de inyección LDAP para:

Acceder a información no autorizada dentro del servidor de directorio.
Modificar datos en el directorio.
Bypass de autenticación para obtener acceso a recursos o cuentas restringidas.
Ejecutar operaciones no deseadas que comprometan la seguridad del sistema.

### Ejemplo de Inyección LDAP

`(&(uid=usuario)(password=clave))`

Si la aplicación no valida correctamente la entrada del usuario, un atacante podría introducir una cadena manipulada como:

`(&(uid=usuario)(&))(password=clave)`

En este caso, la consulta resultante siempre será verdadera, permitiendo que el atacante eluda la autenticación.

### Prevención
Para mitigar los riesgos de una inyección LDAP, es importante:

- Validar y desinfectar las entradas del usuario antes de utilizarlas en consultas LDAP.
- Usar consultas LDAP parametrizadas, lo que reduce la posibilidad de manipulación de los parámetros.
- Restringir los permisos de las aplicaciones en el servidor de directorios, limitando el acceso a solo la información necesaria.

In [2]:
#install ldap3
!pip install ldap3

import re
from ldap3 import Server, Connection, ALL
from ldap3.core.exceptions import LDAPBindError

class LdapHelper:
    # Patrón de expresión regular para validar entradas LDAP
    ldap_safe_pattern = r'^[a-zA-Z0-9\-_]+$'

    # Método para validar la entrada
    @staticmethod
    def is_valid_ldap_input(input_str):
        return re.match(LdapHelper.ldap_safe_pattern, input_str) is not None

    # Método para realizar la búsqueda LDAP
    @staticmethod
    def search_ldap(username, password):
        # Validación de la entrada
        if not LdapHelper.is_valid_ldap_input(username) or not LdapHelper.is_valid_ldap_input(password):
            raise ValueError("Invalid input detected. Input contains unsafe characters.")

        # Configuración de conexión LDAP (ejemplo)
        ldap_path = 'ldap://your-ldap-server.com'
        server = Server(ldap_path, get_info=ALL)
        
        try:
            # Intentar enlazar para validar las credenciales
            conn = Connection(server, user=username, password=password)
            if conn.bind():
                print("User authenticated successfully.")
                conn.unbind()
            else:
                print("Invalid username or password.")
        except LDAPBindError as ex:
            print(f"LDAP error: {ex}")

# Ejemplo de uso
print("Is alumno valid? ", end="")
if LdapHelper.is_valid_ldap_input("alumno"):
    print("Yes")
else:
    print("No")

print("Is 123abc. valid? ", end="")
if LdapHelper.is_valid_ldap_input("123abc."):
    print("Yes")
else:
    print("No")

print("Is user123)(|(uid=*)) valid? ", end="")
if LdapHelper.is_valid_ldap_input("user123)(|(uid=*))"):
    print("Yes")
else:
    print("No")


Is alumno valid? Yes
Is 123abc. valid? No
Is user123)(|(uid=*)) valid? No


### MAS INFORMACION

- https://www.blackhat.com/presentations/bh-europe-08/Alonso-Parada/Whitepaper/bh-eu-08-alonso-parada-WP.pdf
- https://www.exploit-db.com/docs/spanish/21315-[spanish]-taller-de-inyecciones-ldap.pdf
